In [17]:
import sys
sys.path.append('../Utils')

In [18]:
import nltk
import networkx as nx
from collections import defaultdict
from nltk import word_tokenize
from nltk.util import ngrams
import spacy
nltk.download('punkt')
!python -m spacy download es_core_news_sm
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.models import GraphRenderer, StaticLayoutProvider, Circle, MultiLine
from bokeh.plotting import figure
import networkx as nx
import pandas as pd

import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/juandanielcastrellon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 20.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [19]:
from Predict import get_sentiment
from Models import SentimentAnalysisPretrainedBert
from Constants import SEED

In [23]:
# Load the Spanish language model
nlp = spacy.load("es_core_news_sm")

def plural_to_singular(word):
    # Process the word using Spacy
    doc = nlp(word)
    # Return the lemma of the first token
    return doc[0].lemma_

CATEGORIES_DICT = {
    0: 'NEGATIVO',
    1: 'NEUTRO',
    2: 'POSITIVO',
}

#Modelo bert a usar
BERT_MODEL = '../../TonysModels/old-spanish-beto-base-cased.pt'

#Dispositivo sobre el que se corre el modelo de ML
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer_cased = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

torch.manual_seed(SEED)

model = SentimentAnalysisPretrainedBert(BERT_MODEL, 0.4, 4, 256, 128,  True).to(DEVICE)

# Cargar un modelo que ya había sido Finetuneado
checkpoint_classifier = torch.load(
    "../BestModel/Mejores_guardados_beto_cased.pt",
    map_location=torch.device(DEVICE)
)

model.load_state_dict(checkpoint_classifier['model_state_dict'])


<All keys matched successfully>

In [21]:
data = pd.read_csv('../../Resultados/Beto_tagged.csv', sep=';')
columns_to_drop = data.filter(like='Unnamed').columns
data = data.drop(columns=columns_to_drop)
data.head()

,file,Año,Periódico,Ciudad,Texto,Sentimiento,predictions,negative_probs,neutral_probs,positive_probs,ironic_probs,Texto_Lematizado
0,../../Datos/19th_century_Latam_Newspapers_1/PF...,1870,Los loros,Popayán,publicacion semanal ignoramos si sus restos f...,NEUTRO,1.0,9.996460,55.532199,33.521801,0.949539,"['publicacion', 'semanal', 'ignorar', 'resto',..."
1,../../Datos/19th_century_Latam_Newspapers_1/PF...,1870,Los loros,Popayán,se presente en su oficina el que monto en la g...,NEUTRO,1.0,17.339712,63.219225,18.328328,1.112728,"['presentar', 'oficina', 'monto', 'gallina', '..."
2,../../Datos/19th_century_Latam_Newspapers_1/PF...,1870,Los loros,Popayán,"de baron de las tenazas, a don agustin castill...",POSITIVO,2.0,7.059140,42.583779,49.531049,0.826027,"['bar', 'tenaza', 'don', 'agustin', 'castillo'..."
3,../../Datos/19th_century_Latam_Newspapers_1/PF...,1870,Los loros,Popayán,"al articulo presente, en una oracion ferviente...",POSITIVO,2.0,6.416973,21.443181,71.021354,1.118488,"['articulo', 'presente', 'oracion', 'ferviente..."
4,../../Datos/19th_century_Latam_Newspapers_1/PF...,1870,Los loros,Popayán,quien es el que desde aca tantas noticias envi...,NEUTRO,1.0,11.500288,60.579348,27.018249,0.902115,"['aca', 'tanto', 'noticia', 'envia', 'adivinar..."


In [24]:
words = [
    'numero', 'doctor', 'gobierno', 'senor', 'ciudad', 'alli', 'ano', 'bogota', 
    'pais', 'nombre', 'hombre', 'patria', 'dios', 'hombres', 'dia', 'obra', 
    'anos', 'muerte', 'pueblo', 'colombia'
]

len(words)

20

In [27]:
from tqdm import tqdm

def crear_grafo(texts, words_to_search, context=40, num_times=3):
    # Crear un grafo no dirigido
    G = nx.Graph()

    # Preprocesar las palabras para manejar plurales
    words = [plural_to_singular(w) for w in words_to_search]
    words_to_index = {word: i for i, word in enumerate(words)}
    index_to_words = {i: word for i, word in enumerate(words)}

    word_appearences = {}

    edge_sentiment = {}

    node_sentiment = {}
    for text in tqdm(texts):
        tokens = word_tokenize(text.lower())
        # Generar n-gramas de tamaño 'context'

        for i, token in enumerate(tokens):
            min_token = plural_to_singular(token)



            if min_token in words:
                context_min = max(0, i-context)
                context_max = min(len(tokens), i+context)

                context_window = tokens[context_min:context_max]
                sentiment = get_sentiment(model, ' '.join(context_window), tokenizer_cased)[1]
                for word in context_window:
                    min_word = plural_to_singular(word)
                    if min_word in words and min_word!=min_token:
                        if G.has_edge(words_to_index[min_token], words_to_index[min_word]):
                            G[words_to_index[min_token]][words_to_index[min_word]]['weight'] += 1
                        else:
                            G.add_edge(words_to_index[min_token], words_to_index[min_word], weight=1)
                            sent = edge_sentiment.get((min_token, min_word), {})
                            sent[sentiment] = sent.get(sentiment, 0) + 1
                            edge_sentiment[(min_token, min_word)] = sent

                word_appearences[words_to_index[min_token]] = word_appearences.get(words_to_index[min_token], 0) + 1

                if min_token in node_sentiment:
                    node_sentiment[words_to_index[min_token]][sentiment] += 1
                else:
                    node_sentiment[words_to_index[min_token]] = {k: 0 for k in CATEGORIES_DICT.values()}
                    node_sentiment[words_to_index[min_token]][sentiment] += 1

    nx.set_node_attributes(G, word_appearences, 'appearances')

    most_common_sent = {w: max(f, key=f.get) for w, f in node_sentiment.items()}
    nx.set_node_attributes(G, most_common_sent, 'sentiment')

    nx.set_node_attributes(G, index_to_words, 'word')
    for edge, sentiments in edge_sentiment.items():
        most_common_sentiment = max(sentiments, key=sentiments.get)
        G[words_to_index[edge[0]]][words_to_index[edge[1]]]['sentiment'] = most_common_sentiment

    # Filtrar arcos con peso menor a 'num_times'
    for u, v, data in list(G.edges(data=True)):
        G[u][v]['weight'] /= 2
        if data['weight'] < num_times:
            G.remove_edge(u, v)

    return G

G = crear_grafo(list(data.Texto), words, context = 40, num_times = 10)

print("Nodos:", G.nodes(data=True))
print("Arcos:", G.edges(data=True))

 91%|█████████ | 15423/16960 [7:17:23<46:22,  1.81s/it]     

In [26]:
print("Nodos:", len(G.nodes(data=True)))
print("Arcos:", len(G.edges(data=True)))

Nodos: 18
Arcos: 153


In [ ]:
nx.write_gml(G, "grafo/grafo-prueba1-40_contexto-10rep.gml")

In [ ]:
output_notebook()

In [ ]:
from bokeh.plotting import figure, from_networkx, show
from bokeh.models import Legend, LegendItem
from bokeh.models import ColumnDataSource

In [ ]:
words_to_index = {word: i for i, word in enumerate(words)}
index_to_words = {i: word for i, word in enumerate(words)}
def plot_interactive_graph_bokeh(G, node, depth, size_scale=20, edge_scale=3):

    # Extraer el número de apariciones y el sentimiento de los nodos
    appearances = nx.get_node_attributes(G, 'appearances')
    node_sentiments = nx.get_node_attributes(G, 'sentiment')

    # Definir el mapeo de colores para los sentimientos en formato hexadecimal
    sentiment_colors = {
        'POSITIVO': '#4CAF50',  # Verde suave
        'NEGATIVO': '#F44336',  # Rojo suave
        'NEUTRO': '#9E9E9E',    # Gris suave
        'IRONÍA': '#FFEB3B'     # Amarillo suave
    }

    # Obtener los nodos que están a una distancia máxima de 'depth' del nodo en cuestión
    nodes_at_depth = nx.single_source_shortest_path_length(G, node, cutoff=depth).keys()
    subgraph = G.subgraph(nodes_at_depth)

    # Tamaño del gráfico en pulgadas
    width_in_inches = 8
    height_in_inches = 8
    # Convertir a píxeles
    width_in_pixels = width_in_inches * 96
    height_in_pixels = height_in_inches * 96


    # Crear la figura
    p = figure(x_range=(-2, 2), y_range=(-2, 2),
               title="Graph Interaction Demo",
               x_axis_location=None, y_axis_location=None,
               tools="hover,pan,wheel_zoom,reset", tooltips="appearances: @appearances sentiment: @sentiment",
               toolbar_location="above",
               width=width_in_pixels, height=height_in_pixels)
    p.grid.grid_line_color = None

    # Configurar el GraphRenderer
    graph = from_networkx(subgraph, nx.spring_layout, scale=2, center=(0, 0))
    p.renderers.append(graph)

    # Establecer los datos de los nodos y las aristas
    graph.node_renderer.data_source.data['colors'] = [sentiment_colors.get(node_sentiments[node]) for node in subgraph.nodes()]
    graph.node_renderer.data_source.data['size'] = [size_scale * appearances[node] for node in subgraph.nodes()]
    graph.edge_renderer.data_source.data['colors'] = [sentiment_colors.get(G[u][v]['sentiment'], 'black') for u, v in subgraph.edges()]
    graph.edge_renderer.data_source.data['width'] = [edge_scale * G[u][v]['weight'] for u, v in subgraph.edges()]

    graph.node_renderer.glyph.update(size='size', fill_color='colors')
    graph.edge_renderer.glyph.update(line_color='colors', line_width='width')


    x, y = zip(*graph.layout_provider.graph_layout.values())

    print(graph.node_renderer.data_source.data['word'])
    source = ColumnDataSource({'x':x,'y':y,'kid':[ix for ix in graph.node_renderer.data_source.data['word']]})
    labels = LabelSet(x='x', y='y', text='kid', source=source)

    p.renderers.append(labels)

    # Mostrar el gráfico
    show(p)

# Ejemplo de uso
plot_interactive_graph_bokeh(G, words_to_index['tecnología'], 3, size_scale=6, edge_scale=2)


['inteligencia', 'medicina', 'virtual', 'tecnología', 'telemedicina', 'médico', 'dato', 'industria', 'sistema', 'realidad', 'mixto', 'interactivo']


In [8]:
list(data.Texto)

['publicacion semanal  ignoramos si sus restos fueron al fin sepultados nuestros valientes soldados regresaron a sus puestos en ellos estan dispuestos para otro nuevo combate pues tienen en un gran mate abundantes provisiones de chorizos, chicharrones y cascaras de aguacate.  los facciosos emboscados que eran trescientos noventa.  cedieron a la violenta carga de nuestros aliados .  todos fueron amarrados con una cinta amarilla,  , menos jerman vallecilla .  que atropello al centinela por tener dolor de muela debajo de una costilla.  .  .  el estado quedo en paz desde el treinta del pasado.  los loros hemos pensado dar un baile de disfraz dehen llevar antifaz las viejas principalmente para no asustar la gente que ya invitada tenemos, y con la cual beberemos barriles de agua caliente.  varios civicos paseos.  proyectaremos tambien, con el mariscal bacen.  para obsequiar a los fees dos mil quinientos guineos se comera cada uno, mas.  si por desgracia alguno no quedare satisfecho le haremo